In [1]:
%load_ext autoreload
%autoreload 2

%cd '..'

/dss/dsshome1/03/di93fup/polarization_reddit


In [2]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("data/logs/user_partisanship.log"),
        logging.StreamHandler(stream=sys.stdout)
    ]
)

In [3]:
import polars as pl
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

from load.utils import (
    load_df_from_parquet,
    save_df_as_parquet,
)
from preprocessing.utils import (
    calculate_user_party,
)

In [4]:
YEARS = [2015, 2016, 2017, 2018]
YEARS_STR = str(YEARS[0]) if len(YEARS) == 1 else f"{YEARS[0]}-{YEARS[-1]}"
START_MONTH = 1
STOP_MONTH = 12

logging.info(YEARS_STR)


### User Affiliation

In [ ]:
partisan_comments_list = [
    load_df_from_parquet(file_name=f"partisan_comments_{year}.parquet")
    for year in YEARS
]

partisan_comments = pd.concat(partisan_comments_list)

In [ ]:
user_party: pd.DataFrame = partisan_comments.groupby(by="author").progress_apply(
    calculate_user_party,
)  # type: ignore
user_party = user_party[user_party["score"] != 0].reset_index().copy()
user_party["party"] = user_party["party"].astype("string")


In [ ]:
logging.info(user_party.dtypes)

In [ ]:
logging.info(f"Nr of users: {len(user_party)}")

logging.info(user_party.groupby(by="party")["author"].count())

In [ ]:
save_df_as_parquet(
    data=user_party,
    target_file=f"user_party_{YEARS_STR}.parquet",
)

logging.info(user_party.dtypes)
